In [1]:
# libraries
import pandas as pd
import numpy as np
import datetime as dt

import yfinance as yf

import optuna

from sklearn.multioutput import RegressorChain
from sklearn.ensemble import HistGradientBoostingRegressor
from sklearn.model_selection import KFold, cross_validate
from sklearn.metrics import make_scorer, mean_absolute_percentage_error, mean_absolute_error, mean_squared_error

c:\Users\alepi\Documents\projects\personal\repos\toy_ml_pipeline\mlpipe\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# setup
cfg = {
    'algorithm': 'chained_hgb',
    'sklearn_model': HistGradientBoostingRegressor,
    'time_format' : '%Y-%m-%d',
    'date': dt.datetime.today().strftime('%Y-%m-%d'),
    'cv_folds': 4,
    'n_jobs': 4,
    'seed': 42,
    'company':'GOOG',
    'future': 7,
    'past': 90
}

cfg['modelname'] = f"{cfg['algorithm']}_{cfg['date']}"

In [3]:
# dowload stock price data
today = dt.datetime.now()
start = today - dt.timedelta(days=365*10)

df = yf.download(cfg['company'], start = start.strftime(cfg['time_format']), end=today.strftime(cfg['time_format']))
df = df[['Close']].rename(columns={'Close': 'y'})

[*********************100%***********************]  1 of 1 completed


In [4]:
# define and apply window function
def window_df(data: pd.DataFrame, past: int, future: int) -> pd.DataFrame:
    '''Shift and name values in order to have past days of observations as features and future days as targets in a preprocessed dataframe'''

    df = data.copy()
    for i in range(0, past):
        df[f'y_t-{i}'] = df['y'].shift(i)
    for j in range(1, future+1):
        df[f'y_t+{j}'] = df['y'].shift(-j)

    df.drop(columns=['y'], inplace=True)
    df.rename(columns={'y_t-0':'y_t'}, inplace=True)
    #df.dropna(axis=0, inplace=True)
    
    return df

proc_df = window_df(df, past=cfg['past'], future=cfg['future'])

In [5]:
# final preprocessing

# get feature and target names
x_cols = [col for col in proc_df.columns if col.startswith('y_t-') or col=='y_t']
y_cols = [col for col in proc_df.columns if col.startswith('y_t+')]

# save most recent features then remove missing values
x_today = proc_df[x_cols][-1:]
proc_df.dropna(inplace=True)

# pop out last observation as test set
x = proc_df[x_cols][:-1]
y = proc_df[y_cols][:-1]

x_test = proc_df[x_cols][-1:]
y_test = proc_df[y_cols][-1:]

# print info
print(f'Objective: predict {cfg["company"]} stock price {cfg["future"]} days ahead using past {cfg["past"]}-day history')
print(f'Train features range from {x.index[0].strftime(cfg["time_format"])} to {x.index[-1].strftime(cfg["time_format"])} ({x.shape[0]} data points) and targets look {cfg["future"]} steps ahead')
print(f'We predict {cfg["future"]} days ahead using data from {x_test.index[0].strftime(cfg["time_format"])} (we predict this week as a backtesting)')
print('Lastly we will train using all vailable data and make predictions a week from today')

Objective: predict GOOG stock price 7 days ahead using past 90-day history
Train features range from 2013-09-10 to 2023-04-18 (2418 data points) and targets look 7 steps ahead
We predict 7 days ahead using data from 2023-04-19 (we predict this week as a backtesting)
Lastly we will train using all vailable data and make predictions a week from today


In [6]:
# define cross validation strategy
cv = KFold(n_splits = cfg['cv_folds'], shuffle = True, random_state = cfg['seed'])

# fixed params
fixed_params = {
        'loss': 'squared_error',
        'scoring': 'neg_mean_absolute_percentage_error',
        'verbose': 0,
        'early_stopping': True,
        'validation_fraction': .1,
        'n_iter_no_change': 15,
        'random_state': 42 
    }

# objective function for optimization
def objective(trial):
    
    # trial parameters
    tuning_params = {
        'max_iter': trial.suggest_int('max_iter', 100, 1000),
        'learning_rate': trial.suggest_float('learning_rate', 0.05, 0.3),
        'l2_regularization': trial.suggest_float('l2_regularization', 1e-8, 10.0),
        'max_depth': trial.suggest_int('max_depth', 2, 10),
    }
    params = {**fixed_params, **tuning_params}

    # train and score with cv
    model = RegressorChain(base_estimator=HistGradientBoostingRegressor(**params))
    cv_res = cross_validate(
        estimator = model, 
        X = x, 
        y = y,
        scoring = {
            'mape': make_scorer(mean_absolute_percentage_error, greater_is_better=False, multioutput='uniform_average'),
            #'mae': make_scorer(mean_absolute_error, greater_is_better=False, multioutput='uniform_average'),
            #'rmse': make_scorer(mean_squared_error, greater_is_better=False, multioutput='uniform_average', squared=False)
        },
        cv = cv,
        n_jobs= cfg['n_jobs']
    )

    # return mean cv score 
    return -np.mean(cv_res['test_mape'])

In [7]:
# create study
sampler = optuna.samplers.TPESampler(seed=cfg['seed'])
max_trials = 1
time_limit = 3600 * 0.5

study = optuna.create_study(
    sampler=sampler,
    study_name= f"{cfg['modelname']}_optimization",
    direction='minimize')

# perform optimization
print(f"Starting {cfg['modelname']} optimization...")
study.optimize(
    objective,
    n_trials = max_trials,
    timeout = time_limit,
)

[I 2023-05-01 21:27:55,949] A new study created in memory with name: chained_hgb_2023-05-01_optimization


Starting chained_hgb_2023-05-01 optimization...


[I 2023-05-01 21:28:14,355] Trial 0 finished with value: 0.022484966774880228 and parameters: {'max_iter': 437, 'learning_rate': 0.28767857660247903, 'l2_regularization': 7.31993942079411, 'max_depth': 7}. Best is trial 0 with value: 0.022484966774880228.


In [8]:
# optimization results
print(f"Number of finished trials: {len(study.trials)}")
print(f"Best score: {study.best_value}")

best_params = {**fixed_params, **study.best_trial.params}
print("Best trial parameters:")
for k, v in best_params.items():
    print(f"\t{k}: {v}")

# save results
cfg_df = pd.DataFrame([{'algorithm': cfg['algorithm'], 'sklearn_model': cfg['sklearn_model'], 'date': cfg['date'], 'modelname': cfg['modelname'], 'params_names': list(best_params.keys())}])
params_df = pd.DataFrame([best_params])

exp_df = pd.concat([cfg_df, params_df], axis=1)
exp_df.to_csv(f'./{cfg["modelname"]}_exp.csv', index=False)
exp_df

Number of finished trials: 1
Best score: 0.022484966774880228
Best trial parameters:
	loss: squared_error
	scoring: neg_mean_absolute_percentage_error
	verbose: 0
	early_stopping: True
	validation_fraction: 0.1
	n_iter_no_change: 15
	random_state: 42
	max_iter: 437
	learning_rate: 0.28767857660247903
	l2_regularization: 7.31993942079411
	max_depth: 7


,algorithm,sklearn_model,date,modelname,params_names,loss,scoring,verbose,early_stopping,validation_fraction,n_iter_no_change,random_state,max_iter,learning_rate,l2_regularization,max_depth
0,chained_hgb,<class 'sklearn.ensemble._hist_gradient_boosti...,2023-05-01,chained_hgb_2023-05-01,"[loss, scoring, verbose, early_stopping, valid...",squared_error,neg_mean_absolute_percentage_error,0,True,0.1,15,42,437,0.287679,7.319939,7


In [9]:
# load best params
best_params = exp_df[exp_df['params_names'].values[0]].to_dict(orient='records')[0]
print("Final parameters:")
for k, v in best_params.items():
    print(f"\t{k}: {v}")

# backtesting
model = RegressorChain(base_estimator=HistGradientBoostingRegressor(**best_params))
model.fit(x, y)
yp = model.predict(x_test)

# calculate and print metrics
mape = mean_absolute_percentage_error(yp, y_test, multioutput='uniform_average')
mae = mean_absolute_error(yp, y_test, multioutput='uniform_average')
rmse = mean_squared_error(yp, y_test, multioutput='uniform_average', squared=False)
print(f'Average of metric across this weeks predictions:\n\tmape {mape:.4f}\n\tmae {mae:.4f}\n\trmse {rmse:.4f}')

Final parameters:
	loss: squared_error
	scoring: neg_mean_absolute_percentage_error
	verbose: 0
	early_stopping: True
	validation_fraction: 0.1
	n_iter_no_change: 15
	random_state: 42
	max_iter: 437
	learning_rate: 0.28767857660247903
	l2_regularization: 7.31993942079411
	max_depth: 7
Average of metric across this weeks predictions:
	mape 0.0265
	mae 2.7373
	rmse 2.7373


In [10]:
# import matplotlib.pyplot as plt
# import seaborn as sns

# plt.scatter(yp[0], y_test.T.values)

In [11]:
# fit on all data and predict a week from today
x_all, y_all= pd.concat([x, x_test]), pd.concat([y, y_test])

model = RegressorChain(base_estimator=HistGradientBoostingRegressor(**best_params))
model.fit(x, y)

yp = model.predict(x_today)
yp

array([[108.31911676, 108.59025453, 108.49575155, 108.22518639,
        107.6251103 , 109.03460313, 110.41794442]])